In [1]:
import numpy as np
import pandas as pd

This creates data for other years by using functions 

In [ ]:
# %load ../src/functions3.py
import numpy as np
import pandas as pd
visit = pd.read_csv('../data/cleaned_number_of_visitors.csv')
spending = pd.read_csv('../data/cleaned_spending_per_day.csv')
stay = pd.read_csv('../data/cleaned_length_of_stay.csv')
#ata = pd.concat([visit['country'], visit['2022'],spending['2022'], stay['2022']], axis=1)

def remove_total_other(df:pd.DataFrame) -> pd.DataFrame:
	'''
	This function removes total world and something of 'other --' ', but other china is remained.

	Input:pd.DataFrame

	Output:pd.DataFrame  


	''' 
	df2 = df.copy()

	remove_regions = {'Total World', 'Other Countries', 'Rest of the World', 'Other Central & Sth. America', 
	'Other Caribbean', 'Other Africa' 'Other North Africa', 'Rest of Europe' , 'Other Middle East', 'Other North Africa', 'Other Asia'}

	remove2 = {'Other Africa'}

	#for i in len(df):
	#	if df2['country'] in remove_regions:
	#		df2. 
	df2 = df2[~df2['country'].isin(remove_regions )]
	df2 = df2[~df2['country'].isin(remove2 )]





	return df2

def creating_data(year:str):
	'''
	This function creates the data of year

	'''
	data = pd.concat([visit['country'], visit[year],spending[year], stay[year]], axis=1)
	a = year

	b = 'number_of_visit_'+a
	c = 'spending_per_day_'+ a
	d = 'length_of_stay_'+ a
	data.columns = ['country','number_of_visit_'+a, 'spending_per_day_'+ a, 'length_of_stay_'+ a]



	e = 'total_' + year + '_(m)'
	data[e] = data[b]*1000*data[c]*data[d]

	data2 = data.copy()
	data2[e] = data2[e]// 1000000

	#change the unit 
	#data2[e] = data2[e].astype('int64')
	data2[e] = pd.to_numeric(data2[e], errors='coerce').astype('Int64')
	#change the column name 

	name = '../data/visit_spending_length_total' + year + '.csv' 

	data2.to_csv(name, index=False) 





	

	df = data2.copy()
	df = remove_total_other(df)
	data4 = df.reset_index(drop=True)


	# Other china and HongKong is combined. 
	df4 = data4.copy()

	# combine the numerical values
	visit_china = df4.iloc[36,1] + df4.iloc[37,1]
	spend_china = (df4.iloc[36,1]*df4.iloc[36,2] + df4.iloc[37,1]*df4.iloc[37,2])/visit_china  #average
	length_china = (df4.iloc[36,1]*df4.iloc[36,3] + df4.iloc[37,1]*df4.iloc[37,3])/visit_china  #average
	total_china = df4.iloc[36,4] + df4.iloc[37,4]
	china = {'country': ['China'] }


	new_row = pd.DataFrame({'country': 'China',
           b: visit_china ,
           c: round(spend_china,2) ,
          d: int(round(length_china,0)),
          e: total_china},index=[len(df4)])


	# Append the new row to the DataFrame
	df4 = pd.concat([df, new_row])
	df4 = df4.reset_index(drop=True)

	#remove 'Hong Kong' and 'other China'
	data5 = df4.copy()
	data5 = data5.drop(data5[data5['country'].isin(['Hong Kong (China)', 'Other China'])].index)
	data5 = data5.reset_index(drop=True)

	name = '../data/V_S_L_T_by_country' + year + '.csv' 
	data5.to_csv(name, index=False) 

	print("finished")
	



	





In [12]:
%run -i ../src/functions3.py

year = '2018'
creating_data(year)

finished


In [ ]:
# %load ../src/columns_addition.py
import numpy as np
import pandas as pd

def add_columns(data:pd.DataFrame, year:str) -> None:
	'''
	This function add columns according to the year

	Input: 
	pd.DataFrame
	year


	'''

	#eu
	eu = pd.read_excel('../data/eu_countries.xlsx', engine="openpyxl")
	eu = eu[ eu['Name'].isna()==False ]
	data1 = data.copy()
	data1['is_EU'] = data1["country"].apply(lambda x: 'Yes' if x in eu.values else "No")

	
	#Englsih spoken
	# import the data
	english = pd.read_excel('../data/english_spoken.xlsx', engine="openpyxl")
	english_countries = english.iloc[0:44,0]
	data2 = data1.copy()
	data2['english_spoken'] = data2["country"].apply(lambda x: 'Yes' if x in english_countries.values else "No")

	#visa
	# import the data
	visa = pd.read_excel('../data/visa_requirement.xlsx', engine="openpyxl")
	data5 = data2.copy()
	data5['visa_requirement'] = data5["country"].apply(lambda x: 'Yes' if x in visa.values else "No")


	#commonwealth
	# import the data
	commonwealth = pd.read_excel('../data/commonwealth.xlsx', engine="openpyxl")
	data6 = data5.copy()

	# Convert the elements to strings
	commonwealth_array = np.array(commonwealth['Country'], dtype=str)

	# Remove spaces from the elements of the array
	commonwealth_array = np.char.strip(commonwealth_array)


	data6['commonwealth'] = data6["country"].apply(lambda x: 'Yes' if x in commonwealth_array else "No")




	#population
	# download the file
	population = pd.read_excel('../data/population1.xlsx', engine="openpyxl")
	# choose the data
	population1 = population.loc[:,['Country Name', year]]
	population1 = population1.rename(columns={'Country Name':'country', year:'population'})
	population1 = population1[ population1['population'].isna()==False ]
	population1['population'] = population1['population'].astype('int64')
	population1 = population1.reset_index(drop=True)
	#change the unit of population
	population3 = population1.copy()
	population3['population'] = round(population3['population']/1000000,2)

	data8 = data6.copy()

	merged_data =  pd.merge(data8,population3 , on='country', how='left')
	merged_data = merged_data.rename(columns={'population':'population_(m)'})


	#region and income
	# download the file
	region_income = pd.read_excel('../data/region_and_income.xlsx', engine="openpyxl")
	merged_data2 = merged_data.copy()

	#change the column names 
	region_income = region_income.rename(columns={'Country Name':'country', 'Region': 'region', 'IncomeGroup': 'income_group'})
	# add the data
	merged_data3 =  pd.merge(merged_data2,region_income , on='country', how='left')
	

	#GDP
	# download the file
	gdp = pd.read_excel('../data/GDP.xlsx', engine="openpyxl")
	# choose the data
	gdp1 = gdp.loc[:,['country', year]]
	gdp1 = gdp1.rename(columns={year:'gdp'})
	gdp1 = gdp1[ gdp1['gdp'].isna()==False ]
	gdp1['gdp'] = round(gdp1['gdp'],2)
	gdp1 = gdp1.reset_index(drop=True)


	merged_data4 = merged_data3.copy()


	# add the data
	merged_data5 =  pd.merge(merged_data4,gdp1 , on='country', how='left')


	#save the data
	name = '../data/data_' + year + '.csv' 
	merged_data5.to_csv(name, index=False)








In [15]:
#add columns
%run -i ../src/columns_addition.py

year = '2018'
data = pd.read_csv('../data/V_S_L_T_by_country2018.csv')
add_columns(data,year)


In [16]:
#create 2019
%run -i ../src/functions3.py

year = '2019'
creating_data(year)

finished


In [17]:
#add columns
%run -i ../src/columns_addition.py

year = '2019'
data = pd.read_csv('../data/V_S_L_T_by_country2019.csv')
add_columns(data,year)


In [18]:
#create 2021
%run -i ../src/functions3.py

year = '2021'
creating_data(year)

finished


In [19]:
#add columns
%run -i ../src/columns_addition.py

year = '2021'
data = pd.read_csv('../data/V_S_L_T_by_country2021.csv')
add_columns(data,year)
